In [6]:
#libraries and dirrectory 
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import curve_fit
import patientFunctions as ptfn
import seaborn as sns
import matplotlib.pyplot as plt

# directory management

In [7]:
#location of original data
starting_directory = os.getcwd()


machine_directory = 'C:/Users/mcremer' #the C and path to the project folder
# machine_directory = 'C:/Users/maega' #when working from home machine
storage_directory = 'UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects' #Local, HPG, or dropbox
project_directory = 'Cardiac-Amyloidosis-Multiple-Myeloma' #project folder
project_lv2_directory = '012 Processed Data/data-2024-06-05' #deeper part of project folder

path = os.path.join(machine_directory, storage_directory, 
                    project_directory, project_lv2_directory)

new_dirrectory = path

os.chdir(new_dirrectory)

In [8]:
#updates to directory management
machine_directory = 'C:/Users/mcremer' #the C and path to the project folder
# machine_directory = 'C:/Users/maega' #when working from home machine
storage_directory = 'UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects' #Local, HPG, or dropbox
project_directory = 'Cardiac-Amyloidosis-Multiple-Myeloma' #project folder
project_lv2_directory = '015_Mistic2025' #deeper part of project folder

path = os.path.join(machine_directory, storage_directory, 
                    project_directory, project_lv2_directory)

# parent_dir = path
outputDir = path

## naming file outputs

In [9]:
folderNameHeader = "DFsForSVM"
folderNameCommon = ""
folderNameSuffix = "_20250402_v1"

fileNameHeader_1 = "DescFits"
# fileNameHeader_2 = "ALUnknownPts"
fileNameCommon = "_Features"
fileNameSuffix = "_20250402_v1"

folder_data_out = ptfn.makeFolderPathForData(parent_dir=outputDir, folderName_header=folderNameHeader, 
                                        folderName_common=folderNameCommon, folderName_suffix=folderNameSuffix)
# outputToExcel(*, fileName_header=fileNameHeader, fileName_Common= fileNameCommon, 
#               fileName_suffix= fileNameSuffix, folderName = folder_data_out, sheetName= *)

# fig_file_out = outputFiguresPath(fileName_header=fileNameHeader, fileName_mid=**, 
#                                  fileName_suff= fileNameSuffix, parent_dir=parent_dir, folderName=folder_data_out)

folderNameOut = folder_data_out

In [10]:
#file name suffix 
outputSheet_Suffix = fileNameSuffix
outputSheet_common = fileNameCommon

# Penalty scores used throughout for missing data and low time resolution

## length 1

In [11]:
#filler information for fits of 1 point
# one_point_STD = 0
one_point_r2 = -1
# one_point_residual_error = 0

## length 0

In [12]:
penalty_coef= np.nan
penalty_intercept = np.nan
penalty_R2 = np.nan
penalty_residual_error = np.nan
penalty_fillna = np.nan

In [13]:
penalty_echo = -1


# collecting patient information

In [ ]:
#importing patient data as dictionaries of dataframes

#build a dictionary of patients and their values 
ptFiles = ptfn.GetListofPTfiles(new_dirrectory, '.xlsx', 'data')

ptDict = {}

#loop through the listOfPatients 
for file in ptFiles: 
    pt = ptfn.patient(new_dirrectory, file, 4)
    ptDict[pt.ptID] = pt

In [16]:
EchoTimeColumn ='time from diagnosis to first echo (months)'
echoDates = {}
for pt in ptDict.keys():
    ptEchoData = ptDict[pt].echoData
    patientEchoDate = ptEchoData[EchoTimeColumn]
    if 'none' not in list(patientEchoDate):
        echoDates[pt] = patientEchoDate

#test to see if this still runs and if the data is correct

# collecting patients of interest

In [ ]:
#list of patients by amyloid status 

listPositive = []
listNegative = []
listOther = []


list_pts_remove = ["K-01", "G-01", "J-01", "L-02"]
#remove list_pts_remove from ptDict
for pt in list_pts_remove:
    if pt in ptDict:
        del ptDict[pt]

#get list of positive patients 
for pt in ptDict:
    ptALStatus = ptDict[pt].amyloid_status
    if ptALStatus == 'positive':
        listPositive.append(pt)
    elif ptALStatus == 'negative':
        listNegative.append(pt)
    else:
        listOther.append(pt)

# collecting the labs of interest

labs of interest have at least one data point for each patient and aren't tagged as methods or limits of normality

In [ ]:
#for patient labs, convert to boolean and find the number of each lab (true)
ptLabsBool = {}
for pt in ptDict.keys():
    ptLabsData = ptDict[pt].labsData
    #convert to boolean if not nan
    ptLabsData = ptLabsData.notna() #not sure this is capturing the null data correctly
    #find the number of each lab (true)
    ptLabsData = ptLabsData.sum(axis=0) #check the axis
    #add to the dictionary describing the number of each lab the patient has
    ptLabsBool[pt] = ptLabsData

#then find frequency of labs across all patients
ptLabsBool_df = pd.DataFrame(ptLabsBool)
#count frequency of value in a column 

dict_value_counts_labs ={}
for lab in ptLabsBool_df.index:
    #call a row and perform value counts, save it to a key as the lab name in the dictionary
    dict_value_counts_labs[lab] = ptLabsBool_df.loc[lab].value_counts()

#convert to dataframe and transpose
df_value_counts_labs = pd.DataFrame(dict_value_counts_labs).T

In [69]:
#removing things that aren't lab values
omit = ['Method',"Lower Limit of Normal", "Upper Limit of Normal"]

#drop the rows whos string contains items from the omit list
df_value_counts_labs_filtered = df_value_counts_labs[~df_value_counts_labs.index.str.contains('|'.join(omit))]

#collect the rows where the column"0" is null
df_value_counts_labs_filtered = df_value_counts_labs_filtered[df_value_counts_labs_filtered[0].isnull()]

#print the index of df_value_counts_labs_filtered
filteredLabs = df_value_counts_labs_filtered.index.tolist()

In [70]:
filteredLabs

['Unnamed: 0',
 'DeID',
 'ALT Result(U/L)',
 'AST Result(U/L)',
 'Albumin Electrophoresis Result',
 'Albumin Result(gm/dL)',
 'RelTime(Days)',
 'Survival(Mo)',
 'BUN Result(mg/dL)',
 'Beta-2-Microglobulin Result',
 'Calcium Result(mg/dL)',
 'Chloride Result(mmol/L)',
 'Creatinine Result(mg/dL)',
 'Glucose Result',
 'HCT Result',
 'Hemoglobin Result',
 'Immature Gran Auto',
 'Kappa Free Light Chains Result (mg/L)',
 'Kappa/Lambda Free Light Chain Ratio Result',
 'LDH Result(U/L)',
 'Lambda Free Light Chains Result (mg/L)',
 'MCH Result(pg)',
 'MCHC Result',
 'MCV Result',
 'Monocyte Result',
 'Neutrophil Result',
 'Nucleated RBC Result',
 'Pct. Immature Gran Auto Result',
 'Pct. Mono Result',
 'Pct. Neutrophil Result',
 'Platelet Count Result(k/uL)',
 'Potassium Result(mmol/L)',
 'RBC Result',
 'Serum Electrophoresis Alpha 1 Result',
 'Serum Electrophoresis Alpha 2 Result',
 'Serum Electrophoresis Beta Result',
 'Serum Electrophoresis Gamma Result',
 'Sodium Result(mmol/L)',
 'Total Ser

# encoding demographic information

# encoding note information

# descriptive stats for each lab by patient
don't forget to handle instances of 1 for values which aren't counts to be the one recorded value

# fits of data
look in old code to see if there are better ways of writing it.

In [ ]:
#definitions of functions 
# def linear_model(x, a, b):
#     return a * x + b 

def linear_model_forceB(x, a):
    return a * x 

def quadratic_model(x, a, b, c):
    return a * x**2 + b*x + c

# Define model systems
def quadratic_model_force(x, a, b):
    return a * x **2 + b*x 
